## Analisis de Delitos de Ciudad de Buenos Aires - Situación Pandemia

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.graph_objects as go

In [15]:
df = pd.read_csv('datosDelitos.csv',index_col=0, low_memory=False)
df["fecha"] = pd.to_datetime(df["fecha"], infer_datetime_format=True, format='mixed')
df["semana"] = df['fecha'].dt.strftime('%W').astype(int)
df.head(3)

,fecha,franja_horaria,barrio,delitos,id_comuna,anio,mes,dia_semana,horario,semana
0,2016-01-01,13,Parque Avellaneda,Robo,Comuna 9,2016,1,Friday,Tarde,0
1,2016-01-01,22,Palermo,Robo,Comuna 14,2016,1,Friday,Noche,0
2,2016-01-01,1,Parque Patricios,Lesiones por Siniestro Vial,Comuna 4,2016,1,Friday,Madrugada,0


In [72]:
ts = df.groupby(["fecha"], as_index=False)["delitos"].count().set_index('fecha').dropna()


### Serie Temporal
El objetivo es mostrar la cantidad de delitos diarios desde 2016 hasta 2021. Se agrego la media movil de 50 periodos (dias) para visualizar mejor las tendencias.

In [83]:
fig = px.line()

# Calcular la media móvil de 10 periodos
ts['media_movil'] = ts['delitos'].rolling(window=50).mean()

# Agregar la serie de tiempo con transparencia
fig.add_trace(go.Scatter(x=ts.index, y=ts['delitos'], mode='lines', name='Delitos Seria Temporal Diaria',
                        line=dict(color='blue'), opacity=0.4))  # Color azul con transparencia

# Agregar la media móvil con un color distinguible
fig.add_trace(go.Scatter(x=ts.index, y=ts['media_movil'], mode='lines', name='Media Móvil (10 períodos)',
                         line=dict(color='red')))  # Color rojo


# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Fecha', yaxis_title='Cantidad de Delitos',legend=dict(x=0, y=1))
fig.show()

## Analisis Interanual - Mensual
Se realizó una agregación (conteo) mensual y anual para ver el efecto de la pandemia en los años 2020 y 2021.

In [78]:
px.bar(data_frame=df.groupby(["anio","mes"], as_index=False)["delitos"].count().sort_values(["anio", "mes"],  ascending=True),
       x='mes',
       y='delitos', 
       color= "delitos",
       color_discrete_sequence=["red", "green", "blue", "goldenrod", "magenta"],
       facet_col='anio')

In [80]:
px.bar(data_frame=df.loc[df.anio.isin([2019,2020,2021])].groupby(["anio","mes"], 
                                   as_index=False)["delitos"].count().sort_values("delitos", 
                                                                                      ascending=False),
       x='mes',
       y='delitos', 
       color= "delitos",
       color_discrete_sequence=["red", "green", "blue", "goldenrod", "magenta"],
       facet_col='anio')

## Analisis por Barrios y Comunas
Se generó una visualización usando las cantidades de delitos por Comuna y Barrios.

In [98]:
fig = px.treemap(df.groupby(['barrio', 'id_comuna']).count()[['delitos']].reset_index(), 
                 path=['id_comuna', 'barrio'],  # Columna para el treemap
                 values='delitos',  # Valores para el tamaño de los rectángulos
                 color='delitos',  # Color basado en la cantidad de delitos
                 hover_data=['id_comuna', 'delitos'])  # Datos adicionales al pasar el cursor

fig.update_layout(title='Treemap de Delitos por Barrio')
fig.show()